In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import pickle
import os
import numpy as np
import torch
import torch.nn.functional as F
import pickle
import json
import os
from PIL import Image
from pycocotools import mask as coco_mask

from utils.general.dataset_variables import TripletSegmentationVariables 

TRIPLET_DICT = TripletSegmentationVariables.categories['triplet']
# multitask instrument verb and target


## For triplet direct not multitask

In [11]:
# pickle_file_path = '../results/mask2former_direct_triplet_prediction/mask2former_direct_triplet_prediction_best_coco_bbox_mAP_iter_60000.pkl'
# output_json_path  = '../results/mask2former_direct_triplet_prediction/predicted_logits.json'

# pickle_file_path = '../results/full_results_mask2former_multihead/mask2former_one_stage_multihead_full_best_coco_segm_mAP_iter_24500.pkl'
# output_json_path  = '../results/full_results_mask2former_multihead/predicted_logits.json'

pickle_file_path = '../results/triplet_direct_v2/mask2former_test_results.pkl'
output_json_path  = '../results/triplet_direct_v2/predicted_logits.json'


# output_mask_dir = '../../datasets/my_triplet_seg_datasets/triplet_segmentation_dataset_v2_second_stage/test_soft_labels/predicted_instance_masks'

# temperature = 2

In [12]:
with open(pickle_file_path, 'rb') as file:
        # Load the object from the file
        mmdet_results = pickle.load(file)

In [13]:
len(mmdet_results[0]['pred_instances']['logits'][0])

100

In [ ]:
# Initialize output dictionary
final_json = {}

for result  in mmdet_results:
    img_filename = os.path.basename(result ['img_path'])
    img_id = os.path.splitext(img_filename)[0]

    pred_instances = result['pred_instances']
    masks = pred_instances['masks']
    scores = pred_instances['scores']
    logits = pred_instances['logits']
    labels = pred_instances['labels']
    
    # Convert to numpy if needed
    if isinstance(masks, torch.Tensor):
        masks = masks.cpu().numpy()
    if isinstance(logits, torch.Tensor):
        logits = logits.cpu().numpy()
    if isinstance(scores, torch.Tensor):
        scores = scores.cpu().numpy()
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()

    image_instances = []
    
    if isinstance(masks, torch.Tensor):
        masks = masks.cpu().numpy()
    
    label_id_instance_id = {i: 0 for i in range(100)}

    for i, (score, logit, label) in enumerate(zip(scores, logits, labels)):
        if score < 0.5:
            continue
        
        label_id_instance_id[label] += 1
        
        rle = masks[i]
        instance_mask = coco_mask.decode(rle).astype(np.uint8) * 255  # Binary mask
        
        # Remove snare class from logits
        softmax = torch.nn.functional.softmax(torch.tensor(logit), dim=0).numpy()
        predicted_class_name = TRIPLET_DICT[str(label + 1)]

        
        # Save binary mask
        mask_filename = f"{img_id}_{predicted_class_name}_instance_{label_id_instance_id[i]}.png"
        # mask_path = os.path.join(output_mask_dir, mask_filename)
        # Image.fromarray(instance_mask).save(mask_path)
        
        image_instances.append({
            "instance_id": i,
            "predicted_class_name": predicted_class_name,
            "softmax": softmax.tolist(),
            "mask_path": mask_filename
        })
        
    # Save None if no valid detections
    final_json[img_id] = image_instances if image_instances else None
        


# Save
with open(output_json_path, 'w') as f:
    json.dump(final_json, f, indent=4)

print(f"✅ Saved softmax JSON to: {output_json_path}")
# print(f"✅ Masks saved to: {output_mask_dir}")            
        

✅ Saved softmax JSON to: ../results/triplet_direct_v2/predicted_logits.json


## For Multitask

In [2]:
# pickle_file_path = '../results/mask2former_direct_triplet_prediction/mask2former_direct_triplet_prediction_best_coco_bbox_mAP_iter_60000.pkl'
# output_json_path  = '../results/mask2former_direct_triplet_prediction/predicted_logits.json'

# pickle_file_path = '../results/full_results_mask2former_multihead/mask2former_one_stage_multihead_full_best_coco_segm_mAP_iter_24500.pkl'
# output_json_path  = '../results/full_results_mask2former_multihead/predicted_logits.json'

# pickle_file_path = '../results/triplet_direct_v2/mask2former_test_results.pkl'
# output_json_path  = '../results/triplet_direct_v2/predicted_logits.json'

pickle_file_path = '../results/multiple_heads_v2/mask2former_test_results.pkl'
output_json_path  = '../results/multiple_heads_v2/predicted_logits.json'
triplet_map_path="../utils/general/maps.txt"



# output_mask_dir = '../../datasets/my_triplet_seg_datasets/triplet_segmentation_dataset_v2_second_stage/test_soft_labels/predicted_instance_masks'

# temperature = 2

In [3]:
with open(pickle_file_path, 'rb') as file:
        # Load the object from the file
        mmdet_results = pickle.load(file)

In [4]:
# len(mmdet_results[0]['pred_instances']['logits'][0])

mmdet_results[0]['pred_instances'].keys()

dict_keys(['target_labels', 'target_scores', 'bboxes', 'logits', 'labels', 'verb_scores', 'scores', 'masks', 'verb_labels', 'target_logits', 'verb_logits'])

In [5]:
mmdet_results[0]['pred_instances']['scores'].shape

torch.Size([100])

In [6]:
def load_valid_triplets(path="maps.txt"):
    triplets = []
    with open(path, 'r') as f:
        for line in f:
            if line.strip().startswith('# IVT'):
                continue  # skip header
            ivt, i, v, t, _, _ = line.strip().split(',')
            triplets.append((int(ivt), int(i), int(v), int(t)))
    return triplets

In [7]:
def softmax(x):
    x = np.array(x)
    e_x = np.exp(x - np.max(x))  # for numerical stability
    return e_x / e_x.sum()

def generate_softmax_score_for_detected_objects_from_multitask_instrument_verb_and_target_logit(logits_instrument, logits_verb, logits_target, triplet_map):   
    
    instrument_logits = np.array(logits_instrument) 
    verb_logits = np.array(logits_verb)
    target_logits =  np.array(logits_target)
    
    instrument_probs = softmax(instrument_logits)
    verb_probs = softmax(verb_logits)
    target_probs = softmax(target_logits)
    
   
    triplet_softmax = np.zeros(len(triplet_map))
    
    for idx, (ivt, i, v, t) in enumerate(triplet_map):
        triplet_softmax[ivt] = instrument_probs[i] * verb_probs[v] * target_probs[t]
    
    triplet_softmax = triplet_softmax / triplet_softmax.sum()      

    predicted_triplet_index = int(np.argmax(triplet_softmax)) + 1  # if triplet_dict is 1-based
    predicted_triplet_name = TRIPLET_DICT[str(predicted_triplet_index)]
    
    predicted_class_name = predicted_triplet_name.split(',')[0]

    return triplet_softmax, predicted_triplet_name, predicted_class_name

In [10]:
# Initialize output dictionary

from collections import defaultdict

final_json = {}

def generate_predicted_logits_multitask(mmdet_results, output_json_path):

    for result  in mmdet_results:
        img_filename = os.path.basename(result ['img_path'])
        img_id = os.path.splitext(img_filename)[0]

        pred_instances = result['pred_instances']
        masks = pred_instances['masks']
        
        instrument_scores = pred_instances['scores']
        instrument_logits = pred_instances['logits']
        instrument_labels = pred_instances['labels']
        
        verb_scores = pred_instances['verb_scores']
        verb_logits = pred_instances['verb_logits']
        verb_labels = pred_instances['verb_labels']
        
        
        target_scores = pred_instances['target_scores']
        target_logits = pred_instances['target_logits']
        target_labels = pred_instances['target_labels']
        
        # Convert to numpy if needed
        if isinstance(masks, torch.Tensor):
            masks = masks.cpu().numpy()
            
        if isinstance(instrument_logits, torch.Tensor):
            instrument_logits = instrument_logits.cpu().numpy()
        if isinstance(instrument_scores, torch.Tensor):
            instrument_scores = instrument_scores.cpu().numpy()
        if isinstance(instrument_labels, torch.Tensor):
            instrument_labels = instrument_labels.cpu().numpy()
            
        if isinstance(verb_logits, torch.Tensor):
            verb_logits = verb_logits.cpu().numpy()
        if isinstance(verb_scores, torch.Tensor):
            verb_scores = verb_scores.cpu().numpy()
        if isinstance(verb_labels, torch.Tensor):
            verb_labels = verb_labels.cpu().numpy()  
            
        if isinstance(target_logits, torch.Tensor):
            target_logits = target_logits.cpu().numpy()
        if isinstance(target_scores, torch.Tensor):
            target_scores = target_scores.cpu().numpy()
        if isinstance(target_labels, torch.Tensor):
            target_labels = target_labels.cpu().numpy()        

        image_instances = []
        
        if isinstance(masks, torch.Tensor):
            masks = masks.cpu().numpy()
        
        label_name_instance_id = defaultdict(int)

        for i, (instrument_score, verb_score, target_score,
                instrument_logit, verb_logit, target_logit,
                instrument_label, verb_label, target_label) in enumerate(zip(instrument_scores, verb_scores, target_scores,
                                                    instrument_logits, verb_logits, target_logits,
                                                    instrument_labels, verb_labels, target_labels)):
            if instrument_score < 0.5:
                continue
            
            
            
            rle = masks[i]
            # instance_mask = coco_mask.decode(rle).astype(np.uint8) * 255  # Binary mask
            
            # Remove snare class from logits
            # softmax = torch.nn.functional.softmax(torch.tensor(logit), dim=0).numpy()
            # predicted_class_name = TRIPLET_DICT[str(label + 1)]
            
            triplet_map = load_valid_triplets(path=triplet_map_path)
            softmax, predicted_triplet_name, predicted_class_name = generate_softmax_score_for_detected_objects_from_multitask_instrument_verb_and_target_logit(instrument_logit, verb_logit, target_logit,  triplet_map)

            # print(softmax)
            # print(predicted_triplet_name)
            # print(predicted_class_name)
            
            label_name_instance_id[predicted_triplet_name] += 1
            
            # Save binary mask
            mask_filename = f"{img_id}_{predicted_class_name}_instance_{label_name_instance_id[predicted_triplet_name] }.png"
            # mask_path = os.path.join(output_mask_dir, mask_filename)
            # Image.fromarray(instance_mask).save(mask_path)
            
            
            image_instances.append({
                "instance_id": i,
                "predicted_class_name": predicted_triplet_name,
                "softmax": softmax.tolist(),
                "mask_path": mask_filename
            })
            
        # Save None if no valid detections
        final_json[img_id] = image_instances if image_instances else None
            


    # Save
    with open(output_json_path, 'w') as f:
        json.dump(final_json, f, indent=4)

    print(f"✅ Saved softmax JSON to: {output_json_path}")
    # print(f"✅ Masks saved to: {output_mask_dir}")            
        

In [11]:
generate_predicted_logits_multitask(mmdet_results, output_json_path)

✅ Saved softmax JSON to: ../results/multiple_heads_v2/predicted_logits.json
